# Public key distribution
## Motivation

Suppose that our system uses public key cryptography for authentication of data.
Thus, any data sent is signed using the sender's private key.
The receiver can then verify the origin of the data by verifying the signature by checking against the senders public key.

However, notice that the receiver needs a way to obtain the true public key of the sender.

Hence, we need a secure channel for the sender to transmit their public key to the receiver.

## Distribution Methods
### Public announcement
The owner of the public/private key pair can broadcast their public key.
This can be performed by sending a mass email to everyone, or publishing it on a website.

#### Limitations
We still require trust in the public key broadcast channel.
For example, we need to know that the email is not forged, or that the website is not spoofed.

Also, the method of distribution is not standardized, as it can take many different forms.

### Publicly-available directory
The public keys can be stored in a directory on some server.
If a user requires the public key of a certain user, they can query the server to obtain it.

#### Limitations
Once again, the user needs to trust both the server, and the data in the server.
The data in the server may be hard to verify, if the directory is available to be updated by anyone.

Also, requires constant access to the server whenever we need to verify the origin of any messages.

### Public key infrastructure + certificate
**Public key infrastructure (PKI)** is a standardized system of distribution public key.

#### Objectives
* allow large-scale deployment of public key cryptography
* allow public keys to be verified **without** requiring any two communicating parties to **directly** trust each other
* allow managing of keypairs throughout the key's lifecycle.

PKI is based on the following concepts:
* Certificate
* Certificate Authority (CA)

#### Certificate Authority
The CA performs the following:
* issues and signs digital certificates
* keeps a directory of public keys
* has their own public/private keypair
    * the CA's public key has to be securely distributed to all users.

Many OS's and browsers comes preloaded with some CA's public keys.

#### Certificate
A **certificate** is a digital document that contains the following:
* identity of the owner
* public key of the owner
* time period where the certificate is valid
* signature of the CA



#### Verification process
##### Without certificate
Suppose that there is no certificate involved with the PKI system.
Then the process would be as follows:

1. (Bob already knows of the CA's public key)
2. Alice sends Bob a message signed using her private key
3. Bob sends a request to CA about Alice's public key
4. CA respond to Bob, sending Alice's public key, signed using CA's private key
5. Using the **pre-known CA's key**, Bob can verify the message is from CA, thus the public key sent is indeed Alice's.
6. Bob can now verify that the message is from Alice using Alice's public key

Notice that without certificate, this system is very similar to a publicly-available directory of public keys.

##### With certificate
Suppose that there is certificate involved with the PKI system.
Then the process would be as follows:

1. (Bob already knows of the CA's public key)
2. Alice sends Bob a message signed using her private key, along with her certificate signed by the CA
3. Using the pre-known CA's key, Bob can verify the certificate was signed by the CA, thus the public key in the certificate is indeed Alice's.
4. Bob can now verify that the message is from Alice using Alice's public key

Thus, with certificate, the receiver no longer needs to contact the CA whenever they receive an message from an unverified source.

With this system, the CA binds the public key to the entity at the point of verification.
Anyone who trusts the CA will also trust the public key assigned to the identity by the CA.
However, there is still a need for a way to revoke certificates.

#### Identity checking
It is the CA's responsibility to ensure verify that the applicant of the certificate does indeed own the stated identity.

There are 3 tiers of checking for different types of certificates:
1. Domain validation SSL certificate
    * Issued if the applicant can demonstrate the right to administrate the stated domain
2. Organization validation SSL certificate
    * Issued if the applicant has the organization as a legal entity
    * Indicated by a padlock 
3. Extended validation SSL certificate
    * Issued if the applicant has undergone manual identity verification by a human
    * Indicated by a padlock + green address bar in browsers


#### Relationship of trust
##### Types of CA's
There are 3 types of CA's:
* Root CA
    * Certificate is self-signed
    * Can issue certificate to other CA's
* Subordinate/intermediate CA
    * Can issue certificate to other CA's
* Leaf CA
    * Cannot issue certificate to other CA's
    
##### Transfer of trust
Suppose that the certificate of $CA_1$ is issued by the root CA.

1. Alice wishes to send a message to Bob, but her certificate is signed by $CA_1$.
2. Alice, anticipating that Bob may not have the public key of $CA_1$, preemptively sends $CA_1$'s certificate along with her message.
3. Bob, upon reception of the message, can use the root CA's public key to verify that $CA_1$'s certificate is valid.
4. Bob can obtain $CA_1$'s public key from the certificate.
5. Using $CA_1$'s public key, he can now verify that Alice's certificate is signed by $CA_1$.
6. He can now be assured that the message is indeed from Alice by verifying against Alice's public key in Alice's certificate.

(If Alice does not send $CA_1$'s certificate, Bob has to obtain it via some other means)

Notice that with this process, the root CA can allocate the job of signing certificates to an intermediate CA, because the trust in the root CA has been transferred down to the intermediate CA.

##### Certification path/chain
**Certification path/chain** starts with an end-entity certificate, with one or more CA certificates, ending with the root certificate.

Each certificate in the chain is signed by the private key of the next certificate in the chain.
The last certificate (root CA's) is self-signed instead.

The root CA's certificate is known as the **trust anchor**.

#### Certificate revocation
**Non-expired** certificates can be revoked because of:
* private key is compromised
* issuing CA is is compromised
* entity left the organization
* business entity closed

Thus, a verify needs to check if the certificate in question has already been revoked, even if it is yet to be expired.

##### Certification revocation list (CRL)
CA periodically signs and publishes a list of revoked certificates.

Requires constant maintenance from the CA.

##### Online certificate status protocol (OCSP)
An OCSP responder replies whether the certificate is revoked.

###### Issues
* Privacy: OCSP knows the certificates (and thus the websites) that you are verifying
* Soft-fail validation: Some browsers proceed with their communication when there is no response from OCSP

###### Solutions
* OCSP stapling: certificates are accompanied with the OCSP response signed by CA, thus receivers do not need to query OCSP responders for the revocation status
* Incorporate at lower levels: incorporate OCSP as part of TLS handshake

Note that with the above, there will be a drawback of increased network costs.

#### Attacks on PKI
##### Compromised CA

A compromised CA will invalidate the authenticity provided certificates signed by the compromised CA as well as all CA's authorized the compromised CA.

###### Implementation bugs
There are many well known implementation bugs that lead to vulnerabilities.

For example, null-byte injection.
This happens when some browsers truncates the address shown in the address bar when there is a null byte in the address.
However, the full address is used when verifying the certificate.
Thus, a victim being directed to the site www.google.com\0.attacker.com will (incorrectly) see it as www.google.com in their address bar.
Since the attacker's certificate does indeed matches their site of www.google.com\0.attacker.com, there will be no warnings from the browser.
Hence, they will have unknowingly visited the attacker's site.

###### Typo squatting
The attacker can register domains whose names are similar to sites that their victims may use.
For example, they can register and obtain a valid certificate for the site www.goggle.com
Furthermore, they utilize a phishing attack on the victim, sending them the above link.
Even though the address bar shows the address correctly, the attackers are relying on the failure of the victims at checking the address given.